In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam
import numpy as np
import json
import random
import warnings

warnings.filterwarnings('ignore')

# Load intent configuration
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/intent1.json', 'r') as f:
    intents_config = json.load(f)['intents']

# Load menu data
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/mockMenu 1.json', 'r') as f:
    menu_data = json.load(f)

# Data preprocessing
intents = []
unique_intents = []
text_input = []
response_for_intent = {}
query_for_intent = {}

for intent in intents_config:
    intent_name = intent['intent']

    if intent_name not in unique_intents:
        unique_intents.append(intent_name)

    for text in intent['text']:
        text_input.append(text)
        intents.append(intent_name)

    if intent_name not in response_for_intent:
        response_for_intent[intent_name] = []
    for response in intent['responses']:
        response_for_intent[intent_name].append(response)

    if intent_name not in query_for_intent:
        query_for_intent[intent_name] = intent.get('query', '')

tokenizer = Tokenizer(filters='', oov_token='<unk>')
tokenizer.fit_on_texts(text_input)
sequences = tokenizer.texts_to_sequences(text_input)
padded_sequences = pad_sequences(sequences, padding='pre')

intent_to_index = {}
categorical_target = []
index = 0

for intent in intents:
    if intent not in intent_to_index:
        intent_to_index[intent] = index
        index += 1
    categorical_target.append(intent_to_index[intent])

num_classes = len(intent_to_index)
index_to_intent = {index: intent for intent, index in intent_to_index.items()}

categorical_vec = tf.keras.utils.to_categorical(categorical_target, num_classes=num_classes)
categorical_vec = categorical_vec.astype('int32')

# Model parameters
epochs = 100
embed_dim = 300
lstm_num = 50
output_dim = categorical_vec.shape[1]

# Define model
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, embed_dim),
    Bidirectional(LSTM(lstm_num, dropout=0.1)),
    Dense(lstm_num, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    Dense(output_dim, activation='softmax')
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(padded_sequences, categorical_vec, epochs=epochs, verbose=1)

# Predict intent
def predict_intent(sentence):
    sent_tokens = []
    words = sentence.split()
    for word in words:
        if word in tokenizer.word_index:
            sent_tokens.append(tokenizer.word_index[word])
        else:
            sent_tokens.append(tokenizer.word_index.get('<unk>', 0))
    sent_tokens = tf.expand_dims(sent_tokens, 0)
    pred = model(sent_tokens)
    pred_class = np.argmax(pred.numpy(), axis=1)
    intent_name = index_to_intent[pred_class[0]]
    return intent_name

def execute_query(query_code, menu_data):
    try:
        # Ensure proper formatting with indentation
        formatted_query_code = '\n'.join(
            '    ' + line if line.strip() else line for line in query_code.split('\n')
        )

        # Print the formatted query code for debugging
        print("Formatted Query Code:")
        print(formatted_query_code)

        # Define a safe context with local variables
        local_vars = {'menu_data': menu_data, 'query': query, 'item': None, 'result': None}

        exec(f"""
def run_query(menu_data):
    item = None
    {formatted_query_code}
    return item
""", {"__builtins__": None}, local_vars)

        # Call the query function
        result = local_vars['run_query'](menu_data)

        # Check if item is None
        if result is None:
            print("Debug Info: No matching item found in menu_data.")
        else:
            print("Debug Info: Found item:", result)

    except Exception as e:
        print(f"Error executing query: {e}")
        result = "No result found."

    return result



# Generate response
def response(sentence):
    intent_name = predict_intent(sentence)
    intent_config = next((i for i in intents_config if i['intent'] == intent_name), None)

    if intent_config and 'query' in intent_config:
        query_code = intent_config['query']
        # Execute the query
        result = execute_query(query_code, menu_data)
        return result, intent_name
    else:
        return random.choice(response_for_intent.get(intent_name, ["Sorry, I didn't understand the request."])), intent_name

# Test the system
print("Note: Enter 'quit' to break the loop.")
while True:
    query = input('You: ')
    if query.lower() == 'quit':
        break
    bot_response, typ = response(query)
    print(f'Geek: {bot_response} -- TYPE: {typ}')
    print()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0588 - loss: 2.2021
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4118 - loss: 2.1791
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4118 - loss: 2.1501
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2941 - loss: 2.1559
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3529 - loss: 2.1303
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2941 - loss: 2.1565
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3529 - loss: 2.0942
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5294 - loss: 2.0791
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1176 - loss: 2.0940
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4118 - loss: 2.0796
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4706 - loss: 1.9876
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4118 - loss

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam
import numpy as np
import json
import random
import warnings
import time  # Import the time module

warnings.filterwarnings('ignore')

# Load intent configuration
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/intent1.json', 'r') as f:
    intents_config = json.load(f)['intents']

# Load menu data
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/mockMenu 1.json', 'r') as f:
    menu_data = json.load(f)

# Data preprocessing
intents = []
unique_intents = []
text_input = []
response_for_intent = {}
query_for_intent = {}

for intent in intents_config:
    intent_name = intent['intent']

    if intent_name not in unique_intents:
        unique_intents.append(intent_name)

    for text in intent['text']:
        text_input.append(text)
        intents.append(intent_name)

    if intent_name not in response_for_intent:
        response_for_intent[intent_name] = []
    for response in intent['responses']:
        response_for_intent[intent_name].append(response)

    if intent_name not in query_for_intent:
        query_for_intent[intent_name] = intent.get('query', '')

tokenizer = Tokenizer(filters='', oov_token='<unk>')
tokenizer.fit_on_texts(text_input)
sequences = tokenizer.texts_to_sequences(text_input)
padded_sequences = pad_sequences(sequences, padding='pre')

intent_to_index = {}
categorical_target = []
index = 0

for intent in intents:
    if intent not in intent_to_index:
        intent_to_index[intent] = index
        index += 1
    categorical_target.append(intent_to_index[intent])

num_classes = len(intent_to_index)
index_to_intent = {index: intent for intent, index in intent_to_index.items()}

categorical_vec = tf.keras.utils.to_categorical(categorical_target, num_classes=num_classes)
categorical_vec = categorical_vec.astype('int32')

# Model parameters
epochs = 100
embed_dim = 300
lstm_num = 50
output_dim = categorical_vec.shape[1]

# Define model
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, embed_dim),
    Bidirectional(LSTM(lstm_num, dropout=0.1)),
    Dense(lstm_num, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    Dense(output_dim, activation='softmax')
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(padded_sequences, categorical_vec, epochs=epochs, verbose=1)

# Predict intent
def predict_intent(sentence):
    sent_tokens = []
    words = sentence.split()
    for word in words:
        if word in tokenizer.word_index:
            sent_tokens.append(tokenizer.word_index[word])
        else:
            sent_tokens.append(tokenizer.word_index.get('<unk>', 0))
    sent_tokens = tf.expand_dims(sent_tokens, 0)
    pred = model(sent_tokens)
    pred_class = np.argmax(pred.numpy(), axis=1)
    intent_name = index_to_intent[pred_class[0]]
    return intent_name

# Generate response (updated to only return intent)
def response(sentence):
    intent_name = predict_intent(sentence)
    return intent_name

# Test the system with time tracking
print("Note: Enter 'quit' to break the loop.")
while True:
    query = input('You: ')
    if query.lower() == 'quit':
        break

    # Start time tracking
    start_time = time.time()

    # Get the predicted intent
    predicted_intent = response(query)

    # End time tracking
    end_time = time.time()

    # Calculate the time taken
    time_taken = end_time - start_time

    # Print the predicted intent and time taken
    print(f'Predicted Intent: {predicted_intent}')
    print(f'Time taken: {time_taken:.4f} seconds')
    print()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 2.1932
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2353 - loss: 2.1789
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1176 - loss: 2.1734
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5294 - loss: 2.1497
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4118 - loss: 2.1384
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4706 - loss: 2.1182
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4118 - loss: 2.1242
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4118 - loss: 2.0940
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4706 - loss: 2.0755
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4706 - loss: 2.0174
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4706 - loss: 2.0620
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4706 - 

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam
import numpy as np
import json
import pickle

# Load intent configuration
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/intent1.json', 'r') as f:
    intents_config = json.load(f)['intents']

# Load menu data
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/mockMenu 1.json', 'r') as f:
    menu_data = json.load(f)

# Data preprocessing
intents = []
unique_intents = []
text_input = []
response_for_intent = {}
query_for_intent = {}

for intent in intents_config:
    intent_name = intent['intent']

    if intent_name not in unique_intents:
        unique_intents.append(intent_name)

    for text in intent['text']:
        text_input.append(text)
        intents.append(intent_name)

    if intent_name not in response_for_intent:
        response_for_intent[intent_name] = []
    for response in intent['responses']:
        response_for_intent[intent_name].append(response)

    if intent_name not in query_for_intent:
        query_for_intent[intent_name] = intent.get('query', '')

tokenizer = Tokenizer(filters='', oov_token='<unk>')
tokenizer.fit_on_texts(text_input)
sequences = tokenizer.texts_to_sequences(text_input)
padded_sequences = pad_sequences(sequences, padding='pre')

intent_to_index = {}
categorical_target = []
index = 0

for intent in intents:
    if intent not in intent_to_index:
        intent_to_index[intent] = index
        index += 1
    categorical_target.append(intent_to_index[intent])

num_classes = len(intent_to_index)
index_to_intent = {index: intent for intent, index in intent_to_index.items()}

categorical_vec = tf.keras.utils.to_categorical(categorical_target, num_classes=num_classes)
categorical_vec = categorical_vec.astype('int32')

# Model parameters
epochs = 100
embed_dim = 300
lstm_num = 50
output_dim = categorical_vec.shape[1]

# Define model
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, embed_dim),
    Bidirectional(LSTM(lstm_num, dropout=0.1)),
    Dense(lstm_num, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    Dense(output_dim, activation='softmax')
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(padded_sequences, categorical_vec, epochs=epochs, verbose=1)

# Save the model and tokenizer to a pickle file
with open('chatbot_model.pkl', 'wb') as f:
    pickle.dump({'model': model, 'tokenizer': tokenizer, 'intent_to_index': intent_to_index, 'index_to_intent': index_to_intent}, f)

# Function to load the model and tokenizer from pickle
def load_model_and_tokenizer(pickle_file):
    with open(pickle_file, 'rb') as f:
        data = pickle.load(f)
        return data['model'], data['tokenizer'], data['intent_to_index'], data['index_to_intent']

# Load model and tokenizer
model, tokenizer, intent_to_index, index_to_intent = load_model_and_tokenizer('chatbot_model.pkl')

# Predict intent
def predict_intent(sentence):
    sent_tokens = []
    words = sentence.split()
    for word in words:
        if word in tokenizer.word_index:
            sent_tokens.append(tokenizer.word_index[word])
        else:
            sent_tokens.append(tokenizer.word_index.get('<unk>', 0))
    sent_tokens = tf.expand_dims(sent_tokens, 0)
    pred = model(sent_tokens)
    pred_class = np.argmax(pred.numpy(), axis=1)
    intent_name = index_to_intent[pred_class[0]]
    return intent_name

# Generate response (updated to only return intent)
def response(sentence):
    intent_name = predict_intent(sentence)
    return intent_name

# Test the system
print("Note: Enter 'quit' to break the loop.")
while True:
    query = input('You: ')
    if query.lower() == 'quit':
        break

    # Get the predicted intent
    predicted_intent = response(query)

    # Print the predicted intent
    print(f'Predicted Intent: {predicted_intent}')
    print()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.1765 - loss: 2.1978
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4706 - loss: 2.1807
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3529 - loss: 2.1671
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4118 - loss: 2.1432
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2941 - loss: 2.1473
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4118 - loss: 2.1266
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4118 - loss: 2.1107
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3529 - loss: 2.0835
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3529 - loss: 2.0849
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3529 - loss: 2.0803
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3529 - loss: 2.0496
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2941 - loss

# Query Response

In [1]:
import json
from transformers import T5ForConditionalGeneration, T5Tokenizer
from rank_bm25 import BM25Okapi
import torch
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam
import numpy as np
import random

# Load intent configuration
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/intent1.json', 'r') as f:
    intents_config = json.load(f)['intents']

# Load menu data
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/mockMenu 1.json', 'r') as f:
    menu_data = json.load(f)['menu']

# Prepare the corpus for BM25 by combining 'itemName' and 'description' fields
corpus = [
    (item['itemName'] + " " + (item.get('description', '') if item.get('description') else ""))
    for item in menu_data
]
bm25 = BM25Okapi([doc.split() for doc in corpus])

# Data preprocessing for intent classification
intents = []
unique_intents = []
text_input = []
response_for_intent = {}
query_for_intent = {}

for intent in intents_config:
    intent_name = intent['intent']

    if intent_name not in unique_intents:
        unique_intents.append(intent_name)

    for text in intent['text']:
        text_input.append(text)
        intents.append(intent_name)

    if intent_name not in response_for_intent:
        response_for_intent[intent_name] = []
    for response in intent['responses']:
        response_for_intent[intent_name].append(response)

    if intent_name not in query_for_intent:
        query_for_intent[intent_name] = intent.get('query', '')

tokenizer = Tokenizer(filters='', oov_token='<unk>')
tokenizer.fit_on_texts(text_input)
sequences = tokenizer.texts_to_sequences(text_input)
padded_sequences = pad_sequences(sequences, padding='pre')

intent_to_index = {}
categorical_target = []
index = 0

for intent in intents:
    if intent not in intent_to_index:
        intent_to_index[intent] = index
        index += 1
    categorical_target.append(intent_to_index[intent])

num_classes = len(intent_to_index)
index_to_intent = {index: intent for intent, index in intent_to_index.items()}

categorical_vec = tf.keras.utils.to_categorical(categorical_target, num_classes=num_classes)
categorical_vec = categorical_vec.astype('int32')

# Model parameters
epochs = 100
embed_dim = 300
lstm_num = 50
output_dim = categorical_vec.shape[1]

# Define model
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, embed_dim),
    Bidirectional(LSTM(lstm_num, dropout=0.1)),
    Dense(lstm_num, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    Dense(output_dim, activation='softmax')
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(padded_sequences, categorical_vec, epochs=epochs, verbose=1)

# Predict intent
def predict_intent(sentence):
    sent_tokens = []
    words = sentence.split()
    for word in words:
        if word in tokenizer.word_index:
            sent_tokens.append(tokenizer.word_index[word])
        else:
            sent_tokens.append(tokenizer.word_index.get('<unk>', 0))
    sent_tokens = tf.expand_dims(sent_tokens, 0)
    pred = model(sent_tokens)
    pred_class = np.argmax(pred.numpy(), axis=1)
    intent_name = index_to_intent[pred_class[0]]
    return intent_name

# BM25 Document Retrieval
def retrieve_document(query):
    tokenized_query = query.split()
    doc_scores = bm25.get_scores(tokenized_query)
    top_doc_index = torch.argmax(torch.tensor(doc_scores)).item()
    retrieved_item = menu_data[top_doc_index]
    return retrieved_item

# Function Definitions
def get_spicy_dishes_for_fever():
    return [item['itemName'] for item in menu_data if 'spicy' in (item.get('description', '') or '').lower()]

def get_kids_friendly_dishes():
    return [item['itemName'] for item in menu_data if item.get('kidsFriendly') == True]

def get_vegan_dishes():
    vegan_dishes = []
    for item in menu_data:
        # Check if the item has a filter and search for 'Vegan'
        if 'itemFilter' in item:
            for filter_item in item['itemFilter']:
                if filter_item['name'].lower() == 'vegan':
                    vegan_dishes.append(item['itemName'])
                    break  # No need to check other filters if 'Vegan' is found
    return vegan_dishes

# Return nut-free dish names
def get_nut_free_dishes():
    return [item['itemName'] for item in menu_data if 'nuts' not in f"{item.get('description', '')} {item.get('allergicInfo', '')}".lower()]

# Return fish-free dish names
def get_fish_free_dishes():
    return [item['itemName'] for item in menu_data if 'fish' not in f"{item.get('description', '')} {item.get('allergicInfo', '')}".lower()]

def find_min_prep_time_dish_by_subcategory(subcategory):
    # Ensure subcategory is not None and handle cases where subCategory may be None
    filtered_items = [
        item for item in menu_data 
        if item.get('subCategory') and item.get('subCategory', '').lower() == subcategory.lower()
    ]
    if filtered_items:
        return min(filtered_items, key=lambda x: int(x['prepTimeInMins']))['itemName']
    return None


def retrieve_dish_description(query):
    item = retrieve_document(query)
    return item['description'] if item else 'Description not available.'

def retrieve_dish_allergic_info(query):
    item = retrieve_document(query)
    return item['allergicInfo'] if item else 'Allergic info not available.'

def retrieve_dish_price(query):
    item = retrieve_document(query)
    return item['price'] if item else 'Price not available.'

def handle_intent(intent, query):
    switcher = {
        "GetSpicyDishesForFever": get_spicy_dishes_for_fever,
        "GetKidsFriendlyDishes": get_kids_friendly_dishes,
        "GetVeganDishes": get_vegan_dishes,
        "GetNutFreeDishes": get_nut_free_dishes,
        "GetFishFreeDishes": get_fish_free_dishes,
        "FindDishWithLeastPrepTime": find_min_prep_time_dish_by_subcategory,
        "RetrieveDishDescription": retrieve_dish_description,
        "RetrieveDishAllergicInfo": retrieve_dish_allergic_info,
        "RetrieveDishPrice": retrieve_dish_price,
    }

    func = switcher.get(intent)
    if func:
        # Handle cases where subcategory is needed
        if intent == "FindDishWithLeastPrepTime":
            subcategory = query.split('for')[-1].strip() if 'for' in query else None
            return func(subcategory) if subcategory else "Please specify a subcategory."
        # Pass the query for functions that need it (like RetrieveDishPrice)
        elif intent in ["RetrieveDishDescription", "RetrieveDishAllergicInfo", "RetrieveDishPrice"]:
            return func(query)
        # For other functions like GetNutFreeDishes and GetVeganDishes, return the item names directly
        else:
            items = func()
            if isinstance(items, list):  # Ensure it's a list
                return items  # Return the list of item names directly
            return "No items found."
    else:
        return "Invalid intent."

# Execute query for specific intent
def execute_query(intent_name, query):
    if intent_name in query_for_intent:
        query_code = query_for_intent[intent_name]
        # Process the query_code directly if it's valid (consider adapting this to your actual code execution logic)
        return handle_intent(intent_name, query)
    return "Invalid intent or no associated query."

# Generate response
def response(sentence):
    intent_name = predict_intent(sentence)
    intent_config = next((i for i in intents_config if i['intent'] == intent_name), None)

    if intent_config and 'query' in intent_config:
        result = execute_query(intent_name, sentence)
        return result, intent_name
    else:
        if 'description' in sentence.lower():
            return retrieve_dish_description(sentence), intent_name
        elif 'price' in sentence.lower():
            return retrieve_dish_price(sentence), intent_name
        else:
            # Handle specific intent cases
            result = handle_intent(intent_name, sentence)
            return result, intent_name

# Test the system
print("Note: Enter 'quit' to break the loop.")
while True:
    query = input('You: ')
    if query.lower() == 'quit':
        break
    bot_response, typ = response(query)
    print(f'Bot: {bot_response} -- TYPE: {typ}')
    print()

/Users/yash/Library/Python/3.12/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1343 - loss: 2.1979
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1802 - loss: 2.1794 
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2400 - loss: 2.1648 
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2686 - loss: 2.1453 
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2820 - loss: 2.1155 
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2507 - loss: 2.1069 
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2491 - loss: 2.0830 
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2888 - loss: 2.0475 
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3135 - loss: 1.9938 
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3307 - loss: 1.9649 
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3525 - loss: 1.8971 
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3867 - loss

In [1]:
import json
from transformers import T5ForConditionalGeneration, T5Tokenizer
from rank_bm25 import BM25Okapi
import torch
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam
import numpy as np
import random

# Load intent configuration
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/intent1.json', 'r') as f:
    intents_config = json.load(f)['intents']

# Load menu data
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/mockMenu 1.json', 'r') as f:
    menu_data = json.load(f)['menu']

# Prepare the corpus for BM25 by combining 'itemName' and 'description' fields
corpus = [
    (item['itemName'] + " " + (item.get('description', '') if item.get('description') else ""))
    for item in menu_data
]
bm25 = BM25Okapi([doc.split() for doc in corpus])

# Data preprocessing for intent classification
intents = []
unique_intents = []
text_input = []
response_for_intent = {}
query_for_intent = {}

for intent in intents_config:
    intent_name = intent['intent']

    if intent_name not in unique_intents:
        unique_intents.append(intent_name)

    for text in intent['text']:
        text_input.append(text)
        intents.append(intent_name)

    if intent_name not in response_for_intent:
        response_for_intent[intent_name] = []
    for response in intent['responses']:
        response_for_intent[intent_name].append(response)

    if intent_name not in query_for_intent:
        query_for_intent[intent_name] = intent.get('query', '')

tokenizer = Tokenizer(filters='', oov_token='<unk>')
tokenizer.fit_on_texts(text_input)
sequences = tokenizer.texts_to_sequences(text_input)
padded_sequences = pad_sequences(sequences, padding='pre')

intent_to_index = {}
categorical_target = []
index = 0

for intent in intents:
    if intent not in intent_to_index:
        intent_to_index[intent] = index
        index += 1
    categorical_target.append(intent_to_index[intent])

num_classes = len(intent_to_index)
index_to_intent = {index: intent for intent, index in intent_to_index.items()}

categorical_vec = tf.keras.utils.to_categorical(categorical_target, num_classes=num_classes)
categorical_vec = categorical_vec.astype('int32')

# Model parameters
epochs = 100
embed_dim = 300
lstm_num = 50
output_dim = categorical_vec.shape[1]

# Define model
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, embed_dim),
    Bidirectional(LSTM(lstm_num, dropout=0.1)),
    Dense(lstm_num, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    Dense(output_dim, activation='softmax')
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(padded_sequences, categorical_vec, epochs=epochs, verbose=1)

# Predict intent
def predict_intent(sentence):
    sent_tokens = []
    words = sentence.split()
    for word in words:
        if word in tokenizer.word_index:
            sent_tokens.append(tokenizer.word_index[word])
        else:
            sent_tokens.append(tokenizer.word_index.get('<unk>', 0))
    sent_tokens = tf.expand_dims(sent_tokens, 0)
    pred = model(sent_tokens)
    pred_class = np.argmax(pred.numpy(), axis=1)
    intent_name = index_to_intent[pred_class[0]]
    return intent_name

# BM25 Document Retrieval
def retrieve_document(query):
    tokenized_query = query.split()
    doc_scores = bm25.get_scores(tokenized_query)
    top_doc_index = torch.argmax(torch.tensor(doc_scores)).item()
    retrieved_item = menu_data[top_doc_index]
    return retrieved_item

# Function Definitions
def get_spicy_dishes_for_fever():
    return [item['itemName'] for item in menu_data if 'spicy' in (item.get('description', '') or '').lower()]

def get_kids_friendly_dishes():
    return [item['itemName'] for item in menu_data if item.get('kidsFriendly') == True]

def get_vegan_dishes():
    vegan_dishes = []
    for item in menu_data:
        if 'itemFilter' in item:
            for filter_item in item['itemFilter']:
                if filter_item['name'].lower() == 'vegan':
                    vegan_dishes.append(item['itemName'])
                    break
    return vegan_dishes

def get_nut_free_dishes():
    return [item['itemName'] for item in menu_data if 'nuts' not in f"{item.get('description', '')} {item.get('allergicInfo', '')}".lower()]

def get_fish_free_dishes():
    return [item['itemName'] for item in menu_data if 'fish' not in f"{item.get('description', '')} {item.get('allergicInfo', '')}".lower()]

def find_min_prep_time_dish_by_subcategory(subcategory):
    filtered_items = [
        item for item in menu_data 
        if item.get('subCategory') and item.get('subCategory', '').lower() == subcategory.lower()
    ]
    if filtered_items:
        return min(filtered_items, key=lambda x: int(x['prepTimeInMins']))['itemName']
    return None

def retrieve_dish_description(query):
    item = retrieve_document(query)
    return item['description'] if item else 'Description not available.'

def retrieve_dish_allergic_info(query):
    item = retrieve_document(query)
    return item['allergicInfo'] if item else 'Allergic info not available.'

def retrieve_dish_price(query):
    item = retrieve_document(query)
    return item['price'] if item else 'Price not available.'

# Modified handle_intent to return natural language responses
def handle_intent(intent, query):
    switcher = {
        "GetSpicyDishesForFever": get_spicy_dishes_for_fever,
        "GetKidsFriendlyDishes": get_kids_friendly_dishes,
        "GetVeganDishes": get_vegan_dishes,
        "GetNutFreeDishes": get_nut_free_dishes,
        "GetFishFreeDishes": get_fish_free_dishes,
        "FindDishWithLeastPrepTime": find_min_prep_time_dish_by_subcategory,
        "RetrieveDishDescription": retrieve_dish_description,
        "RetrieveDishAllergicInfo": retrieve_dish_allergic_info,
        "RetrieveDishPrice": retrieve_dish_price,
    }

    func = switcher.get(intent)
    if func:
        # Handle cases where subcategory is needed
        if intent == "FindDishWithLeastPrepTime":
            subcategory = query.split('for')[-1].strip() if 'for' in query else None
            if subcategory:
                dish = func(subcategory)
                if dish:
                    return f"The dish in the {subcategory} category that takes the least time to prepare is {dish}.", intent
                else:
                    return f"Sorry, I couldn't find any dish in the {subcategory} category.", intent
            else:
                return "Please specify a subcategory, so I can help you find the dish with the least preparation time.", intent
        # Pass the query for functions that need it (like RetrieveDishPrice)
        elif intent in ["RetrieveDishDescription", "RetrieveDishAllergicInfo", "RetrieveDishPrice"]:
            result = func(query)
            if intent == "RetrieveDishDescription":
                return f"The description of the dish is: {result}.", intent
            elif intent == "RetrieveDishAllergicInfo":
                return f"The allergic information for the dish is: {result}.", intent
            elif intent == "RetrieveDishPrice":
                return f"The price of the dish is {result}.", intent
        # For other functions like GetNutFreeDishes and GetVeganDishes
        else:
            items = func()
            if isinstance(items, list) and items:
                item_list = ', '.join(items)
                return f"Here are some options: {item_list}.", intent
            else:
                return "Sorry, I couldn't find any relevant items.", intent
    else:
        return "I'm not sure how to help with that.", intent

# Modify the response generation to be more conversational
def response(sentence):
    intent_name = predict_intent(sentence)
    intent_config = next((i for i in intents_config if i['intent'] == intent_name), None)

    if intent_config and 'query' in intent_config:
        result, intent = execute_query(intent_name, sentence)
        return f"I think you're asking about {intent_name}. {result}", intent_name
    else:
        if 'description' in sentence.lower():
            result = retrieve_dish_description(sentence)
            return f"Let me tell you about this dish. {result}", intent_name
        elif 'price' in sentence.lower():
            result = retrieve_dish_price(sentence)
            return f"You might be wondering about the cost. The price is {result}.", intent_name
        else:
            # Handle specific intent cases
            result, intent = handle_intent(intent_name, sentence)
            if result:
                return f"It seems like you're asking about {intent_name}. {result}", intent_name
            else:
                return "I'm not sure I understand. Could you clarify?", intent_name

# Execute query for specific intent
def execute_query(intent_name, query):
    if intent_name in query_for_intent:
        query_code = query_for_intent[intent_name]
        # Process the query_code directly if it's valid
        return handle_intent(intent_name, query)
    return "It seems like I don't have an answer for that right now.", intent_name

# Test the system with user input
print("Note: Enter 'quit' to exit the conversation.")
while True:
    query = input('You: ')
    if query.lower() == 'quit':
        print("Goodbye! If you need help again, just ask.")
        break
    bot_response, intent_type = response(query)
    print(f'Bot: {bot_response} -- Intent: {intent_type}')
    print()


/Users/yash/Library/Python/3.12/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1070 - loss: 2.1892
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2127 - loss: 2.1775 
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2283 - loss: 2.1630 
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2406 - loss: 2.1454 
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1567 - loss: 2.1449 
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2228 - loss: 2.1082 
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2989 - loss: 2.1020 
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2686 - loss: 2.0539 
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3906 - loss: 1.9905 
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3112 - loss: 1.9555 
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2995 - loss: 1.9003 
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3684 - loss

In [1]:
import json
from rank_bm25 import BM25Okapi
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam

# Load intent configuration
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/intent1.json', 'r') as f:
    intents_config = json.load(f)['intents']

# Load menu data
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/mockMenu 1.json', 'r') as f:
    menu_data = json.load(f)['menu']

# Prepare the corpus for BM25 by combining 'itemName' and 'description' fields
corpus = [
    (item['itemName'] + " " + (item.get('description', '') if item.get('description') else ""))
    for item in menu_data
]
bm25 = BM25Okapi([doc.split() for doc in corpus])

# Data preprocessing for intent classification
text_input = []
intents = []
response_for_intent = {}
query_for_intent = {}

for intent in intents_config:
    intent_name = intent['intent']

    for text in intent['text']:
        text_input.append(text)
        intents.append(intent_name)

    response_for_intent[intent_name] = intent['responses'][0]  # Get the first response template
    query_for_intent[intent_name] = intent.get('query', '')

# Tokenizer and data preparation
tokenizer = Tokenizer(filters='', oov_token='<unk>')
tokenizer.fit_on_texts(text_input)
sequences = tokenizer.texts_to_sequences(text_input)
padded_sequences = pad_sequences(sequences, padding='pre')

# Prepare categorical target
intent_to_index = {intent: index for index, intent in enumerate(set(intents))}
categorical_target = [intent_to_index[intent] for intent in intents]

# One-hot encoding
categorical_vec = tf.keras.utils.to_categorical(categorical_target)

# Define the model
embed_dim = 300
lstm_num = 50
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, embed_dim),
    Bidirectional(LSTM(lstm_num, dropout=0.1)),
    Dense(lstm_num, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    Dense(categorical_vec.shape[1], activation='softmax')
])

# Compile and fit the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(padded_sequences, categorical_vec, epochs=100, verbose=1)

# Predict intent function
def predict_intent(sentence):
    tokens = [tokenizer.word_index.get(word, tokenizer.word_index['<unk>']) for word in sentence.split()]
    sent_tokens = pad_sequences([tokens], padding='pre', maxlen=padded_sequences.shape[1])
    pred = model.predict(sent_tokens)
    pred_class = np.argmax(pred, axis=1)[0]
    return list(intent_to_index.keys())[pred_class]

# Retrieve document function
def retrieve_document(query):
    tokenized_query = query.split()
    doc_scores = bm25.get_scores(tokenized_query)
    top_doc_index = np.argmax(doc_scores)
    return menu_data[top_doc_index]

# Retrieve dish description function
def retrieve_dish_description(query):
    item = retrieve_document(query)
    if item:
        return item['itemName'], item['description']
    return None, 'Description not available.'

# Retrieve allergic information function
def retrieve_dish_allergic_info(query):
    item = retrieve_document(query)
    return item['allergicInfo'] if item else 'Allergic info not available.'

# Retrieve price function
def retrieve_dish_price(query):
    item = retrieve_document(query)
    return item['price'] if item else 'Price not available.'

# Get spicy dishes for fever
def get_spicy_dishes_for_fever():
    return [item['itemName'] for item in menu_data if 'spicy' in (item.get('description', '') or '').lower()]

# Get kids-friendly dishes
def get_kids_friendly_dishes():
    return [item['itemName'] for item in menu_data if item.get('kidsFriendly')]

# Get vegan dishes
def get_vegan_dishes():
    return [item['itemName'] for item in menu_data if 'vegan' in (item.get('itemFilter', '') or '').lower()]

# Get nut-free dishes
def get_nut_free_dishes():
    return [item['itemName'] for item in menu_data if 'nuts' not in f"{item.get('description', '')} {item.get('allergicInfo', '')}".lower()]

# Get fish-free dishes
def get_fish_free_dishes():
    return [item['itemName'] for item in menu_data if 'fish' not in f"{item.get('description', '')} {item.get('allergicInfo', '')}".lower()]

# Find dish with least prep time by subcategory
def find_min_prep_time_dish_by_subcategory(subcategory):
    filtered_items = [
        item for item in menu_data
        if item.get('subCategory') and item.get('subCategory', '').lower() == subcategory.lower()
    ]
    if filtered_items:
        return min(filtered_items, key=lambda x: int(x['prepTimeInMins']))['itemName']
    return None

# Define the RAG operation for all intents
def rag_operation(intent, query):
    response_template = response_for_intent.get(intent, "I'm not sure how to respond.")
    dishes = ""

    if intent == "RetrieveDishDescription":
        dish_name, description = retrieve_dish_description(query)
        if dish_name:
            dishes = f"{dish_name} is described as: {description}"
        else:
            dishes = "Description not available."

    elif intent == "RetrieveDishAllergicInfo":
        allergic_info = retrieve_dish_allergic_info(query)
        dishes = allergic_info

    elif intent == "RetrieveDishPrice":
        price = retrieve_dish_price(query)
        dishes = price

    elif intent == "GetSpicyDishesForFever":
        spicy_dishes = get_spicy_dishes_for_fever()
        dishes = ', '.join(spicy_dishes) if spicy_dishes else "No spicy dishes found."

    elif intent == "GetKidsFriendlyDishes":
        kids_friendly_dishes = get_kids_friendly_dishes()
        dishes = ', '.join(kids_friendly_dishes) if kids_friendly_dishes else "No kids-friendly dishes found."

    elif intent == "GetVeganDishes":
        vegan_dishes = get_vegan_dishes()
        dishes = ', '.join(vegan_dishes) if vegan_dishes else "No vegan dishes found."

    elif intent == "GetNutFreeDishes":
        nut_free_dishes = get_nut_free_dishes()
        dishes = ', '.join(nut_free_dishes) if nut_free_dishes else "No nut-free dishes found."

    elif intent == "GetFishFreeDishes":
        fish_free_dishes = get_fish_free_dishes()
        dishes = ', '.join(fish_free_dishes) if fish_free_dishes else "No fish-free dishes found."

    elif intent == "FindDishWithLeastPrepTime":
        subcategory = query.split('for')[-1].strip() if 'for' in query else None
        if subcategory:
            least_prep_dish = find_min_prep_time_dish_by_subcategory(subcategory)
            dishes = least_prep_dish if least_prep_dish else "No dish found for the specified subcategory."
        else:
            dishes = "Please specify a subcategory."

    # Constructing the final response using the template
    return response_template.format(dishes=dishes)

# Predict intent and respond function
def predict_and_respond(sentence):
    intent_name = predict_intent(sentence)
    
    # Perform RAG operation
    rag_output = rag_operation(intent_name, sentence)

    # Return the final response
    return rag_output, intent_name

# Interactive Chat Loop
print("Note: Enter 'quit' to exit.")
while True:
    user_input = input('You: ')
    if user_input.lower() == 'quit':
        print("Goodbye!")
        break
    bot_response, intent_name = predict_and_respond(user_input)
    print(f'Bot: {bot_response}\nIntent: {intent_name}')

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0963 - loss: 2.2007
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2462 - loss: 2.1781 
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1847 - loss: 2.1603 
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2260 - loss: 2.1406 
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2328 - loss: 2.1370 
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2283 - loss: 2.1076 
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3369 - loss: 2.0750 
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2709 - loss: 2.0516 
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3408 - loss: 1.9880 
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3314 - loss: 1.9692 
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4019 - loss: 1.8983 
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4796 - loss

In [18]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam
from rank_bm25 import BM25Okapi
 
# Load intent configuration
with open('Intent.json', 'r') as f:
    intents_config = json.load(f)['intents']
 
# Load menu data
with open('mockMenu 1.json', 'r') as f:
    menu_data = json.load(f)['menu']
 
# Prepare the corpus for BM25 by combining 'itemName' and 'description' fields
corpus = [
    (item['itemName'] + " " + (item.get('description', '') if item.get('description') else ""))
    for item in menu_data
]
bm25 = BM25Okapi([doc.split() for doc in corpus])
 
# Data preprocessing for intent classification
text_input = []
intents = []
response_for_intent = {}
query_for_intent = {}
 
for intent in intents_config:
    intent_name = intent['intent']
    for text in intent['text']:
        text_input.append(text)
        intents.append(intent_name)
    response_for_intent[intent_name] = intent['responses'][0]  # Get the first response template
    query_for_intent[intent_name] = intent.get('query', '')
 
# Tokenizer and data preparation
tokenizer = Tokenizer(filters='', oov_token='<unk>')
tokenizer.fit_on_texts(text_input)
sequences = tokenizer.texts_to_sequences(text_input)
padded_sequences = pad_sequences(sequences, padding='pre')
 
# Prepare categorical target
intent_to_index = {intent: index for index, intent in enumerate(set(intents))}
index_to_intent = {v: k for k, v in intent_to_index.items()}  # reverse map for predicted intent
categorical_target = [intent_to_index[intent] for intent in intents]
 
# One-hot encoding
categorical_vec = tf.keras.utils.to_categorical(categorical_target)
 
# Define the model
embed_dim = 300
lstm_num = 50
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, embed_dim),
    Bidirectional(LSTM(lstm_num, dropout=0.1)),
    Dense(lstm_num, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    Dense(categorical_vec.shape[1], activation='softmax')
])
 
# Compile and fit the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(padded_sequences, categorical_vec, epochs=100, verbose=1)
 
# Predict intent function
def predict_intent(sentence):
    tokens = [tokenizer.word_index.get(word, tokenizer.word_index['<unk>']) for word in sentence.split()]
    sent_tokens = pad_sequences([tokens], padding='pre', maxlen=padded_sequences.shape[1])
    pred = model.predict(sent_tokens)
    pred_class = np.argmax(pred, axis=1)[0]
    return index_to_intent[pred_class]
 
# Retrieve document function
def retrieve_document(query):
    tokenized_query = query.split()
    doc_scores = bm25.get_scores(tokenized_query)
    top_doc_index = np.argmax(doc_scores)
    return menu_data[top_doc_index] if doc_scores[top_doc_index] > 0 else None
 
# Functions to retrieve specific data from the menu
def retrieve_dish_description(query):
    item = retrieve_document(query)
    if item:
        return item.get('itemName', 'Unknown dish'), item.get('description', 'No description available')
    return None, None
 
def retrieve_dish_allergic_info(query):
    item = retrieve_document(query)
    if item:
        return item.get('itemName', 'Unknown dish'), item.get('allergicInfo', 'No allergic info available')
    return None, None
 
def retrieve_dish_price(query):
    item = retrieve_document(query)
    if item:
        return item.get('itemName', 'Unknown dish'), item.get('price', 0)
    return None, None
 
def get_kids_friendly_dishes():
    return [item for item in menu_data if item.get('kidsFriendly')]
 
def get_spicy_dishes_for_fever():
    return [item for item in menu_data if any(f.get('name', '').lower() == 'spicy' for f in item.get('itemFilter', []))]
 
def get_vegan_dishes():
    return [item for item in menu_data if any(f.get('name', '').lower() == 'vegan' for f in item.get('itemFilter', []))]
 
def get_nut_free_dishes():
    return [item for item in menu_data if any(f.get('name', '').lower() == 'nut-free' for f in item.get('itemFilter', []))]
 
def get_fish_free_dishes():
    return [item for item in menu_data if 'fish' not in item.get('allergicInfo', '').lower()]
 
def find_dish_with_least_prep_time():
    return min(menu_data, key=lambda x: x.get('prepTime', float('inf')))
 
# RAG operation that fetches dish names or other info based on the intent
def rag_operation(intent, query):
    response_template = response_for_intent.get(intent, "I'm not sure how to respond.")
    dishes = ""
 
    # Handle the intents dynamically
    intent_functions = {
        "GetKidsFriendlyDishes": get_kids_friendly_dishes,
        "GetSpicyDishesForFever": get_spicy_dishes_for_fever,
        "GetVeganDishes": get_vegan_dishes,
        "GetNutFreeDishes": get_nut_free_dishes,
        "GetFishFreeDishes": get_fish_free_dishes,
        "FindDishWithLeastPrepTime": find_dish_with_least_prep_time,
        "RetrieveDishDescription": retrieve_dish_description,
        "RetrieveDishAllergicInfo": retrieve_dish_allergic_info,
        "RetrieveDishPrice": retrieve_dish_price
    }
 
    if intent in intent_functions:
        if intent in ["RetrieveDishDescription", "RetrieveDishAllergicInfo", "RetrieveDishPrice"]:
            item_name, result = intent_functions[intent](query)
            if item_name:
                if intent == "RetrieveDishPrice":
                    dishes = response_template.format(dish_name=item_name, price=result)
                elif intent == "RetrieveDishAllergicInfo":
                    dishes = response_template.format(dish_name=item_name, allergic_info=result)
                else:
                    dishes = response_template.format(dish_name=item_name, description=result)
            else:
                dishes = "Information not available."
        else:
            dishes_list = intent_functions[intent]()
            dishes = ', '.join([item['itemName'] for item in dishes_list]) or "No dishes available."
    else:
        dishes = "I'm not sure how to respond to that."
 
    return dishes
 
# Response construction: links intent, query, and RAG operation
def response(sentence):
    # Step 1: Predict intent
    intent_name = predict_intent(sentence)
    
    # Step 2: Perform RAG (Retrieve Augment Generate) operation
    rag_output = rag_operation(intent_name, sentence)
 
    return rag_output, intent_name
 
# Interactive Chat Loop
print("Note: Enter 'quit' to exit.")
while True:
    user_input = input('You: ')
    if user_input.lower() == 'quit':
        print("Goodbye!")
        break
    bot_response, intent_name = response(user_input)
    print(f'Bot: {bot_response}\nIntent: {intent_name}')

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0690 - loss: 2.1947
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.1724 - loss: 2.1820
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2759 - loss: 2.1750
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3103 - loss: 2.1571
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2759 - loss: 2.1473
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2759 - loss: 2.1275
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3103 - loss: 2.1270
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3103 - loss: 2.1065
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3448 - loss: 2.0818
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3103 - loss: 2.0705
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3793 - loss: 2.0513
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2414 - loss

TypeError: string indices must be integers, not 'str'

In [2]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util
from fuzzywuzzy import process
import re
 
# Load intent configuration
with open('Intent 1.json', 'r') as f:
    intents_config = json.load(f)['intents']
 
# Load menu data
with open('mockMenu 1.json', 'r') as f:
    menu_data = json.load(f)['menu']
 
# Prepare the corpus for BM25 by combining 'itemName' and 'description' fields
corpus = [
    (item['itemName'] + " " + (item.get('description', '') if item.get('description') else ""))
    for item in menu_data
]
bm25 = BM25Okapi([doc.split() for doc in corpus])
 
# Prepare dish names for keyword extraction
dish_names = [item['itemName'] for item in menu_data]
 
# Load Sentence Transformer model
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
 
# Data preprocessing for intent classification
text_input = []
intents = []
response_for_intent = {}
query_for_intent = {}
 
for intent in intents_config:
    intent_name = intent['intent']
    for text in intent['text']:
        text_input.append(text)
        intents.append(intent_name)
    response_for_intent[intent_name] = intent['responses'][0]  # Get the first response template
    query_for_intent[intent_name] = intent.get('query', '')
 
# Tokenizer and data preparation
tokenizer = Tokenizer(filters='', oov_token='<unk>')
tokenizer.fit_on_texts(text_input)
sequences = tokenizer.texts_to_sequences(text_input)
padded_sequences = pad_sequences(sequences, padding='pre')
 
# Prepare categorical target
intent_to_index = {intent: index for index, intent in enumerate(set(intents))}
index_to_intent = {v: k for k, v in intent_to_index.items()}  # reverse map for predicted intent
categorical_target = [intent_to_index[intent] for intent in intents]
 
# One-hot encoding
categorical_vec = tf.keras.utils.to_categorical(categorical_target)
 
# Define the model
embed_dim = 300
lstm_num = 50
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, embed_dim),
    Bidirectional(LSTM(lstm_num, dropout=0.1)),
    Dense(lstm_num, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    Dense(categorical_vec.shape[1], activation='softmax')
])
 
# Compile and fit the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(padded_sequences, categorical_vec, epochs=100, verbose=1)
 
# Predict intent function
def predict_intent(sentence):
    tokens = [tokenizer.word_index.get(word, tokenizer.word_index['<unk>']) for word in sentence.split()]
    sent_tokens = pad_sequences([tokens], padding='pre', maxlen=padded_sequences.shape[1])
    pred = model.predict(sent_tokens)
    pred_class = np.argmax(pred, axis=1)[0]
    return index_to_intent[pred_class]
 
def extract_dish_name(query):
    query = query.lower()
    best_match = process.extractOne(query, dish_names)
    if best_match[1] > 80:  # You can set a threshold for matching score
        return best_match[0]
    return None
 
# Retrieve document function with BM25 only
def retrieve_document(query):
    dish_name = extract_dish_name(query)
    if not dish_name:
        return None
    # Get BM25 scores
    tokenized_query = dish_name.split()
    doc_scores = bm25.get_scores(tokenized_query)
    top_doc_index = np.argmax(doc_scores)
    
    return menu_data[top_doc_index] if doc_scores[top_doc_index] > 0 else None
 
# Functions to retrieve specific data from the menu
def retrieve_dish_description(query):
    item = retrieve_document(query)
    if item:
        return item.get('itemName', 'Unknown dish'), item.get('description', 'No description available')
    return None, None
 
def retrieve_dish_allergic_info(query):
    item = retrieve_document(query)
    if item:
        return item.get('itemName', 'Unknown dish'), item.get('allergicInfo', 'No allergic info available')
    return None, None
 
def retrieve_dish_price(query):
    item = retrieve_document(query)
    if item:
        return item.get('itemName', 'Unknown dish'), item.get('price', 0)
    return None, None
 
def get_kids_friendly_dishes():
    return [item for item in menu_data if item.get('kidsFriendly')]
 
def get_spicy_dishes_for_fever():
    return [item for item in menu_data if any(f.get('name', '').lower() == 'spicy' for f in item.get('itemFilter', []))]
 
def get_vegan_dishes():
    return [item for item in menu_data if any(f.get('name', '').lower() == 'vegan' for f in item.get('itemFilter', []))]
 
def get_nut_free_dishes():
    return [item for item in menu_data if any(f.get('name', '').lower() == 'nut-free' for f in item.get('itemFilter', []))]
 
def get_fish_free_dishes():
    return [item for item in menu_data if 'fish' not in item.get('allergicInfo', '').lower()]
 
def find_dish_with_least_prep_time():
    return min(menu_data, key=lambda x: x.get('prepTime', float('inf')))
 
# RAG operation that fetches dish names or other info based on the intent
def rag_operation(intent, query):
    response_template = response_for_intent.get(intent, "I'm not sure how to respond.")
    dishes = ""
 
    # Handle the intents dynamically
    intent_functions = {
        "GetKidsFriendlyDishes": get_kids_friendly_dishes,
        "GetSpicyDishesForFever": get_spicy_dishes_for_fever,
        "GetVeganDishes": get_vegan_dishes,
        "GetNutFreeDishes": get_nut_free_dishes,
        "GetFishFreeDishes": get_fish_free_dishes,
        "FindDishWithLeastPrepTime": find_dish_with_least_prep_time,
        "RetrieveDishDescription": retrieve_dish_description,
        "RetrieveDishAllergicInfo": retrieve_dish_allergic_info,
        "RetrieveDishPrice": retrieve_dish_price
    }
 
    if intent in intent_functions:
        if intent in ["RetrieveDishDescription", "RetrieveDishAllergicInfo", "RetrieveDishPrice"]:
            item_name, result = intent_functions[intent](query)
            if item_name:
                if intent == "RetrieveDishPrice":
                    dishes = response_template.format(dish_name=item_name, price=result)
                elif intent == "RetrieveDishAllergicInfo":
                    dishes = response_template.format(dish_name=item_name, allergic_info=result)
                else:
                    dishes = response_template.format(dish_name=item_name, description=result)
            else:
                dishes = "Information not available."
        else:
            dishes_list = intent_functions[intent]()
            dishes = ', '.join([item['itemName'] for item in dishes_list]) or "No dishes available."
    else:
        dishes = "I'm not sure how to respond to that."
 
    return dishes
 
# Response construction: links intent, query, and RAG operation
def response(sentence):
    # Step 1: Predict intent
    intent_name = predict_intent(sentence)
    
    # Step 2: Perform RAG (Retrieve Augment Generate) operation
    rag_output = rag_operation(intent_name, sentence)
 
    return rag_output, intent_name
 
# Interactive Chat Loop
print("Note: Enter 'quit' to exit.")
while True:
    user_input = input('You: ')
    if user_input.lower() == 'quit':
        print("Goodbye!")
        break
    bot_response, intent_name = response(user_input)
    print(f'Bot: {bot_response}\nIntent: {intent_name}')

/opt/homebrew/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/opt/homebrew/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/Users/yash/Library/Python/3.12/lib/python/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1034 - loss: 2.1913
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1034 - loss: 2.1816
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2069 - loss: 2.1717
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1724 - loss: 2.1516
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2759 - loss: 2.1377
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2759 - loss: 2.1268
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3448 - loss: 2.1176
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2414 - loss: 2.0912
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2069 - loss: 2.0965
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2414 - loss: 2.0712
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2069 - loss: 2.0405
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3103 - loss